# Vector Arithmetic #

One of the features that Qahirah adds on top of Cairo is the ability to perform arithmetic on both x and y coordinates at the same time. This is in addition to the ability to transform vectors by using matrices; that is useful too, but some calculations are easier to express just using `Vector` objects.

In [ ]:
import math
from ipywidgets.widgets import \
    interact
import ipywidgets.widgets as \
    widgets
from IPython.display import \
    display_png
import qahirah as qah
from qahirah import \
    CAIRO, \
    Colour, \
    Matrix, \
    Path, \
    Rect, \
    Vector

pix = qah.ImageSurface.create \
  (
    format = CAIRO.FORMAT_RGB24,
    dimensions = (400, 400)
  )
ctx = None

def reset() :
    "(re)initializes the drawing context, wiping out any existing drawing."
    global ctx
    del ctx
    ctx = qah.Context.create(pix)
    (ctx
        .save()
        .set_source_colour(Colour.grey(.95))
        .paint()
        .restore()
    )
#end reset

def display() :
    "(re)displays what has been drawn."
    display_png(pix.to_png_bytes(), raw = True)
#end display

reset()


For a simple example, consider measuring a layout using units other than the default Cairo units. These are normally interpreted as 1/72 inch (though this can be changed with appropriate transformation matrices). What if you wanted to operate in, say, millimetres? Simply define an appropriate unit multiplier:

    mm = 72 / 25.4

Now you can define, for example, a page size like A4, which is 210mm×297mm, like this:

    a4_page_size = Vector(210, 297) * mm

which is a bit more convenient than having to write the values out separately:

    a4_page_width = 210 * mm
    a4_page_height = 297 * mm

For another example, consider the common problem of laying out some figures on a 2D grid. We might have parameters like a *margin* gap between figures (and also around the figures), the numbers of *rows* and *columns* of figures to draw plus the dimensions of the figure itself. The code for doing the layout in C might look like this:

    for (row = 0; row < nr_rows; ++row)
      {
        for (col = 0; col < nr_cols; ++col)
          {
            cairo_save(ctx);
            float x_offset =
                col * (margin_x + figure_width) + margin_x;
            float y_offset =
                row * (margin_y + figure_height) + margin_y;
            cairo_translate(ctx, x_offset, y_offset);
            ... do drawing ...
            cairo_restore(ctx);
          } /*for*/
      } /*for*/

whereas in Qahirah, we can directly do the calculations on `Vector` objects, perhaps defined like this:

    margin = Vector(margin_x, margin_y)
    figure_size = Vector(figure_width, figure_height)

and then the layout loop becomes a little simpler:

    for row in range(nr_rows) :
        for col in range(nr_cols) :
            ctx.save()
            ctx.translate \
              (
                  Vector(col, row) * (margin + figure_size) + margin
              )
            ... do drawing ...
            ctx.restore()
        #end for
    #end for

Consider a related calculation: given the bounds for the figure and the page size, how many instances of the figure can we fit across and down? In C, the calculation might look like

    nr_cols = floor(page_width / figure_width);
    nr_rows = floor(page_height / figure_height);

whereas in Python with Qahirah we can just do

    grid = math.floor(page_size / figure_size)

where `grid.x` is the number of columns, and `grid.y` is the number of rows.

Here is an interactive example: try different values for the page and figure dimensions, and watch how the layout adapts.

In [ ]:
@interact \
  (
    page_w = (20, 400, 20),
    page_h = (20, 400, 20),
    fig_w = (10, 100, 10),
    fig_h = (10, 100, 10),
  )
def grid_fit(page_w, page_h, fig_w, fig_h) :
    page_size = Vector(page_w, page_h)
    fig_size = Vector(fig_w, fig_h)
    margin = Vector(10, 10)
    grid = math.floor((page_size - margin) / (fig_size + margin))
    reset()
    ctx.source_colour = Colour.from_hsva((0.2, 0.5, 0.9))
    ctx.line_width = 3
    ctx.rectangle(Rect.from_dimensions(page_size))
    ctx.stroke()
    ctx.source_colour = Colour.from_hsva((0.45, 0.5, 0.7))
    for row in range(grid.y) :
        for col in range(grid.x) :
            ctx.save()
            ctx.translate(Vector(col, row) * (fig_size + margin) + margin)
            ctx.rectangle(Rect.from_dimensions(fig_size))
            ctx.fill()
            ctx.restore()
        #end for
    #end for
    display()
#end grid_fit

### Fitting Within Bounds ###

Consider another common problem: positioning a figure centred within given bounds, scaling it down if necessary. Quite often the aspect ratio of the figure might not match those of the bounding area, so to avoid distortion or clipping, you have to compute the scaling factors both horizontally and vertically, and use the lesser of the two, then compute the necessary offset to centre the figure within the bounds. In C, this calculation might look like

    scale_x = fmin(figure_width / bounds_x, 1.0);
    scale_y = fmin(figure_height / bounds_y, 1.0);
    scale = fmin(scale_x, scale_y);
    offset_x = (bounds_x - figure_x * scale) / 2.0;
    offset_y = (bounds_y - figure_y * scale) / 2.0;

whereas in Python with Qahirah, this can be done much more simply as

    scale = min(min(bounds / figure_size), 1.0)
    offset = (bounds - figure_size * scale) / 2.0

Note the expression `min(bounds / figure_size)` in the above: applying the built-in `min()` function to a `Vector` returns the smaller of the x or y coordinates; similarly `max()` returns the larger of the two.


In [ ]:
def make_test_figure(figure_bounds) :
    figure = qah.ImageSurface.create \
      (
        format = CAIRO.FORMAT_ARGB32,
        dimensions = round(figure_bounds)
      )
    ctx = \
        (qah.Context.create(figure)
            .set_source_colour(Colour.grey(0, 0))
            .paint()
            .new_path()
            .append_path
              (
                Path.create_arc
                  (
                    centre = figure_bounds / 2,
                    radius = figure_bounds.y / 2,
                    angle1 = 0,
                    angle2 = qah.circle,
                    negative = False,
                    closed = True
                  ).transform
                    (
                      Matrix.scale
                        (
                          factor = (figure_bounds.x / figure_bounds.y, 1),
                          centre = figure_bounds / 2
                        )
                    )
              )
            .clip()
        )
    bounds = Rect(0, 0, figure_bounds.x, figure_bounds.y / 3)
    for colour, y_offset in \
        (
            (Colour.from_hsva((0, 0.5, 0.7)), 0),
            (Colour.from_hsva((1 / 3, 0.5, 0.7)), figure_bounds.y / 3),
            (Colour.from_hsva((2 / 3, 0.5, 0.7)), 2 * figure_bounds.y / 3),
        ) \
    :
        (ctx
            .new_path()
            .rectangle(bounds + Vector(0, y_offset))
            .set_source_colour(colour)
            .fill()
        )
    #end for
    figure.flush()
    return \
        figure
#end make_test_figure

@interact \
  (
    outer_w = (20, 400, 20),
    outer_h = (20, 400, 20),
    inner_w = (20, 400, 20),
    inner_h = (20, 400, 20),
  )
def fit_within_bounds(outer_w, outer_h, inner_w, inner_h) :
    outer = Vector(outer_w, outer_h)
    inner = Vector(inner_w, inner_h)
    scale = min(min(outer / inner), 1.0)
    offset = (outer - inner * scale) / 2.0
    reset()
    ctx.source_colour = Colour.from_hsva((0.13, 0.9, 0.9))
    ctx.rectangle(Rect.from_dimensions(outer))
    ctx.fill()
    if False :
        # just a simple rectangle for inner figure
        ctx.source_colour = Colour.from_hsva((0.6, 0.9, 0.9))
        ctx.rectangle(Rect.from_dimensions(inner * scale) + offset)
        ctx.stroke()
    else :
        # more elaborate inner figure
        ctx.source = \
          (
            qah.Pattern.create_for_surface(make_test_figure(inner))
                .set_matrix
                  (
                    ~(
                          Matrix.translate(offset)
                      *
                          Matrix.scale(scale)
                    )
                  )
          )
        ctx.paint()
    #end if
    display()
#end fit_within_bounds